In [1]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='raman-461708')

glance_dataset = ee.FeatureCollection(
  "projects/sat-io/open-datasets/GLANCE/GLANCE_TRAINING_DATA_V1"
)

roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

In [2]:

# Filter: Level 1 Class ID = 6  (choose whichever class you want)
filtered = glance_dataset.filter(
    ee.Filter.eq('Glance_Class_ID_level1', 6)
).filterBounds(roi)  # Increase or decrease as needed

crop_filtered = glance_dataset.filter(
    ee.Filter.eq('Glance_Class_ID_level2',12 )
).filterBounds(roi)  # Increase or decrease as needed
# Create an interactive map
Map = geemap.Map(center=[20.59, 78.96], zoom=5)  # India view
url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
Map.layout.height = '1000px'
Map.add_tile_layer(url, name="Google Map", attribution="Google")

# Add filtered samples to map
Map.addLayer(
    filtered,
    {'color': 'red'},
    'GLANCE Class 6 Samples'
)

Map.addLayer(
    crop_filtered,
    {'color': 'green'},
    'GLANCE Class 2 Samples'
)

# Display
Map

Map(center=[20.59, 78.96], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [4]:
# ---------------------------
# 1. Load your LULC image
# ---------------------------
lulc = ee.Image(
    "projects/raman-461708/assets/LULC_v4_PanIndia_2023-07-01_2024-06-30"
)

# Your class mapping:
# crops = 8, 9, 10, 11  → label 1
# scrub = 12            → label 2
lulc_2class = (
    lulc
    .remap([0,1,2,3,4,5,6,7,8, 9, 10, 11, 12,13], [0,0,0,0,0,0,0,0,1, 1, 1, 1, 2,0])
    .rename('pred_label')  # rename the band for clarity
)

# ---------------------------
# 2. Load GLANCE reference
# ---------------------------
glance = ee.FeatureCollection(
    "projects/sat-io/open-datasets/GLANCE/GLANCE_TRAINING_DATA_V1"
).filterBounds(roi)

# !!! SET THESE LISTS ACCORDING TO GLANCE LEGEND !!!
# Example (you MUST correct these based on the actual GLANCE classes):
CROP_GLANCE_CLASSES = [1]   # placeholder
SCRUB_GLANCE_CLASSES = [3]  # placeholder

# Crop reference points
glance_crop = (
    glance
    .filter(ee.Filter.inList('Glance_Class_ID_level2', [12]))
    .map(lambda f: f.set('ref_label', 1))
)

# Scrub reference points
glance_scrub = (
    glance
    .filter(ee.Filter.inList('Glance_Class_ID_level1', [6]))
    .map(lambda f: f.set('ref_label', 2))
)

# Merge crop + scrub reference
ref_points = glance_crop.merge(glance_scrub)


# ---------------------------
# 3. Sample your LULC at reference points
# ---------------------------
samples = lulc_2class.sampleRegions(
    collection=ref_points,
    properties=['ref_label'],  # keep the reference label
    scale=10,
    geometries=False
)

# ---------------------------
# 4. Compute confusion matrix & accuracies
# ---------------------------
conf_matrix = samples.errorMatrix('ref_label', 'pred_label')

print('Confusion Matrix:\n', conf_matrix.getInfo())
print('Overall Accuracy:', conf_matrix.accuracy().getInfo())
print('Producer\'s Accuracy:', conf_matrix.producersAccuracy().getInfo())
print('Consumers Accuracy (User\'s):', conf_matrix.consumersAccuracy().getInfo())

Confusion Matrix:
 [[0, 0, 0], [196, 2023, 74], [389, 24, 62]]
Overall Accuracy: 0.7532514450867052
Producer's Accuracy: [[0], [0.8822503270824248], [0.13052631578947368]]
Consumers Accuracy (User's): [[0, 0.9882755251587689, 0.45588235294117646]]


In [4]:
print('Consumers Accuracy (User\'s):', conf_matrix.consumersAccuracy().getInfo())

Consumers Accuracy (User's): [[0, 0.9882755251587689, 0.45588235294117646]]


In [7]:
dw_ic = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
         .filterDate('2023-07-01', '2024-06-30')
         .filterBounds(roi))

# Take mode of the "label" band over time (most frequent class)
dw_mode = (dw_ic
           .select('label')
           .mode()
           .rename('dw_label'))

# Map DW classes to your 2-class scheme:
# 4 (crops) → 1 (crop)
# 5 (shrub & scrub) → 2 (scrub)
dw_2class = (dw_mode
             .remap([4, 5], [1, 2], defaultValue=0)
             .rename('pred_label_dw'))

# Sample DW at GLANCE reference points
samples_dw = dw_2class.sampleRegions(
    collection=ref_points,
    properties=['ref_label'],  # true label: 1=crop, 2=scrub
    scale=10,
    geometries=False
)

# Keep only samples where DW actually predicted crop or scrub (1 or 2)
samples_dw = samples_dw.filter(ee.Filter.neq('pred_label_dw', 0))
conf_dw = samples_dw.errorMatrix('ref_label', 'pred_label_dw')

print('Dynamic World – Confusion Matrix:\n', conf_dw.getInfo())
print('Dynamic World – Overall Accuracy:', conf_dw.accuracy().getInfo())
print('Dynamic World – Producer\'s Accuracy:', conf_dw.producersAccuracy().getInfo())
print('Dynamic World – Consumer\'s Accuracy (User\'s):', conf_dw.consumersAccuracy().getInfo())


EEException: Computation timed out.

In [8]:
task = ee.batch.Export.table.toDrive(
    collection=conf_dw,
    description='conf_dw',
    fileFormat='CSV'
)
task.start()

In [5]:
# ---------------------------------
# 1. ESA WorldCover (v200)
# ---------------------------------
# WorldCover v200 has an ImageCollection with YEAR property; select 2021
wc_ic = ee.ImageCollection("ESA/WorldCover/v200")

wc_img = wc_ic.first().select('Map').rename('wc_raw')

# Map WorldCover → your 2 classes:
# 40 (Cropland)   → 1 (crop)
# 20 (Shrubland)  → 2 (scrub)
# everything else → 0 (other)
wc_2class = (wc_img
             .remap([40, 20, 10, 30, 50,60,70,80,90, 95, 100],   # from
                    [1, 2, 0,0,0,0,0,0,0,0,0],     # to
                    defaultValue=0)
             .rename('pred_label_wc'))
wc_2class = wc_2class.clip(roi)
# Sample WorldCover prediction at GLANCE reference points
samples_wc = wc_2class.sampleRegions(
    collection=ref_points,
    properties=['ref_label'],      # true labels from GLANCE
    scale=10,
    geometries=False
)

# Keep only samples where WorldCover actually predicted crop or scrub (1 or 2)
samples_wc = samples_wc.filter(ee.Filter.neq('pred_label_wc', 0))


In [6]:
conf_wc = samples_wc.errorMatrix('ref_label', 'pred_label_wc')

print('ESA WorldCover – Confusion Matrix:\n', conf_wc.getInfo())
print('ESA WorldCover – Overall Accuracy:', conf_wc.accuracy().getInfo())
print('ESA WorldCover – Producer\'s Accuracy:', conf_wc.producersAccuracy().getInfo())
print('ESA WorldCover – Consumer\'s Accuracy (User\'s):', conf_wc.consumersAccuracy().getInfo())


ESA WorldCover – Confusion Matrix:
 [[0, 0, 0], [0, 1961, 19], [0, 13, 249]]
ESA WorldCover – Overall Accuracy: 0.9857270294380018
ESA WorldCover – Producer's Accuracy: [[0], [0.9904040404040404], [0.950381679389313]]
ESA WorldCover – Consumer's Accuracy (User's): [[0, 0.9934143870314083, 0.9291044776119403]]


In [13]:
samples_wc

In [8]:
wri_data = ee.FeatureCollection("projects/raman-461708/assets/wri_india_data")

In [ ]:
wri_data.filter(ee.Filter.eq("class", 2))

In [13]:
Map.addLayer(
    wri_data.filter(ee.Filter.eq("class", 2)),
    {'color': 'blue'},
    'WRI Class 2 Samples'
)
Map

Map(bottom=4117.0, center=[20.59, 78.96], controls=(WidgetControl(options=['position', 'transparent_bg'], posi…

In [15]:
var = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
var

In [17]:
# Choose the year you want (example: 2019)
lc_2019 = var.first()

# Add to map with default visualization
Map.addLayer(lc_2019, {}, 'ESA WorldCover 100m (2019)')

Map

Map(bottom=115643.0, center=[21.362292063619908, 75.79605102539064], controls=(WidgetControl(options=['positio…

In [4]:
import ee
import geemap


ee.Authenticate()
ee.Initialize(project='raman-461708')
# Initialize the Earth Engine API (run ee.Authenticate() once before this in a new environment)

# Base path for your assets
BASE = "projects/raman-461708/assets/"

# Filenames you listed (with .tif as on disk)
# In GEE, asset IDs will typically *not* have the .tif extension, so we'll strip it.
names = [
    "label_dw_84p9880584034_27p0039414850-20190327.tif",
    "label_dw_76p0227625521_13p6112592251-20190422.tif",
    "label_dw_72p5923588818_34p2048014672-20191011.tif",
    "label_dw_84p9135464396_28p2423059406-20190501.tif",
    "label_dw_89p5106586237_23p3347134686-20191014.tif",
    "label_dw_94p4985339347_27p0498068352-20190119.tif",
    "label_dw_72p1668513125_35p1221084713-20190613.tif",
    "label_dw_81p0430339698_29p2972089953-20190429.tif",
    "label_dw_80p2873123112_29p2054782949-20190427.tif",
    "label_dw_85p0142951733_21p0414459584-20190309.tif",
    "label_dw_75p6978027949_32p3701874590-20181219.tif",
    "label_dw_85p1580394604_26p6828840336-20190327.tif",
    "label_dw_80p2906159963_13p1067403729-20190511.tif",
    "label_dw_72p8314397499_19p1609665999-20190329.tif",
    "label_dw_88p1352904041_26p6828840336-20181231.tif",
    "label_dw_71p5110571120_33p9754747162-20191011.tif",
    "label_dw_73p1149788229_33p9754747162-20191028.tif",
    "label_dw_85p4153719505_26p8204800842-20190327.tif",
    "label_dw_77p8319653978_11p8225105671-20190330.tif",
    "label_dw_84p6960154003_27p9671138393-20190506.tif",
    "label_dw_81p3382966252_28p1964405904-20181230.tif",
    "label_dw_72p2700233150_22p0963490131-20191015.tif",
    "label_dw_70p8228304348_33p7461479652-20191021.tif",
    "label_dw_73p1786808577_33p7920133154-20191028.tif",
    "label_dw_74p3755860693_33p1957637627-20191008.tif",
    "label_dw_79p9700062424_29p0678822443-20181218.tif",
    "label_dw_82p1709093201_18p7023130979-20190317.tif",
    "label_dw_81p4236952477_16p4090455876-20190101.tif",
    "label_dw_79p5320063247_28p9761515439-20181218.tif",
    "label_dw_79p4895668827_29p6641317969-20181218.tif",
    "label_dw_87p3491492766_26p7287493838-20190202.tif",
    "label_dw_70p7175323092_22p6467332155-20191008.tif",
    "label_dw_80p2301892962_8p0615518503-20181217.tif",
    "label_dw_81p2733633308_28p6550940924-20181230.tif",
    "label_dw_76p3674443319_16p3173148872-20190402.tif",
    "label_dw_76p2701350463_31p3611497545-20190331.tif",
    "label_dw_79p4808318425_18p7481784481-20190325.tif",
    "label_dw_79p3310738461_10p6300114618-20190401.tif",
    "label_dw_75p1276684753_32p1408607080-20181219.tif",
    "label_dw_77p3214471124_12p3270294194-20190325.tif",
]

# Build list of ee.Image objects
images = []
for n in names:
    asset_id = BASE + n.replace(".tif", "")
    images.append(ee.Image(asset_id))

# Create an ImageCollection from the list
ic = ee.ImageCollection(images)

# Mosaic to display as one layer (you can also just add single tiles if you prefer)
lulc_mosaic = ic.select("b2").mosaic()
label_mosaic = ic.select("b1").mosaic()

# Create a geemap map
m = geemap.Map(center=[22, 80], zoom=4)

# Simple palettes; adjust min/max according to your class range
lulc_vis = {
    "min": 0,
    "max": 10,
    "palette": [
        "000000",  # 0 = no markup
        "1f77b4",  # 1
        "ff7f0e",  # 2
        "2ca02c",  # 3
        "d62728",  # 4
        "9467bd",  # 5
        "8c564b",  # 6
        "e377c2",  # 7
        "7f7f7f",  # 8
        "bcbd22",  # 9
        "17becf",  # 10
    ],
}

label_vis = {
    "min": 0,
    "max": 10,
    "palette": [
        "ffffff",  # 0
        "1f77b4",
        "ff7f0e",
        "2ca02c",
        "d62728",
        "9467bd",
        "8c564b",
        "e377c2",
        "7f7f7f",
        "bcbd22",
        "17becf",
    ],
}

url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")

m.addLayer(lulc_mosaic, lulc_vis, "LULC (annotated)")
m.addLayer(label_mosaic, label_vis, "Top-1 predicted label")

# Display the map (in a Jupyter notebook)
m


Map(center=[22, 80], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

In [13]:
DW_labels = ee.ImageCollection("projects/wri-datalab/dynamic_world/v1/DW_LABELS")

In [14]:
# The point of interest
point = ee.Geometry.Point([22.66, 70.71])

# Filter collection by intersection
intersecting = DW_labels.filterBounds(m.draw_features[0].geometry())
intersecting

In [15]:
ic.filterBounds(m.draw_features[0].geometry())

In [16]:
DW_labels.limit(1)

In [17]:
m.addLayer(intersecting, lulc_vis, "DW")

In [18]:
m

Map(bottom=913711.0, center=[22.62494441438227, 70.75504302978517], controls=(WidgetControl(options=['position…

In [19]:
lulc_test = lulc_mosaic
lulc_ours = ee.Image(
    "projects/raman-461708/assets/LULC_v4_PanIndia_2023-07-01_2024-06-30"
)

In [21]:
ref = lulc_test

# Keep only classes 5 and 6
ref_masked = ref.updateMask(ref.eq(5).Or(ref.eq(6)))

# -----------------------------------------
# STEP 2: Remap our classes into test schema
# -----------------------------------------
# Our croplands = 8,9,10,11 → map to 5
# Our scrub = 12 → map to 6

ours = lulc_ours.select("predicted_label")

remapped = ours.remap(
    [8, 9, 10, 11, 12],   # from
    [5, 5, 5,  5,  6],    # to (crop=5, scrub=6)
    0                     # default
)

# Mask prediction wherever reference is not valid
pred_masked = remapped.updateMask(ref_masked.mask())

# -----------------------------------------
# STEP 3: Stack reference + prediction
# -----------------------------------------
stack = pred_masked.rename("pred").addBands(ref_masked.rename("ref"))

# -----------------------------------------
# STEP 4: Sample the stack to a FeatureCollection
# -----------------------------------------
# Choose an appropriate scale for your LULC (10 or 30 m depending on product)
SCALE = 10  # adjust if your LULC is 30 m

# Use the geometry of the reference as region
region = ref.geometry()

samples = stack.sample(
    region=region,
    scale=SCALE,
    numPixels=1_000_000,  # cap to avoid insane collections; adjust as needed
    geometries=False,
    tileScale=4           # helps with large areas
)

# -----------------------------------------
# STEP 5: Compute confusion matrix
# -----------------------------------------
cm = samples.errorMatrix("ref", "pred")

print("Confusion Matrix (ref vs pred):")
print(cm.getInfo())

print("\nOverall Accuracy:")
print(cm.accuracy().getInfo())

print("\nProducer's Accuracy (per ref class):")
print(cm.producersAccuracy().getInfo())

print("\nUser's Accuracy (per pred class):")
print(cm.consumersAccuracy().getInfo())

Confusion Matrix (ref vs pred):


EEException: Image.sample: Provide 'geometry' parameter when aggregating over an unbounded image.